[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/XPretrain-colab/blob/main/test.ipynb)

In [ ]:
!wget https://raw.githubusercontent.com/microsoft/XPretrain/main/hd-vila-100m/src/cut_videos.py -O /content/cut_videos.py
!wget https://raw.githubusercontent.com/microsoft/XPretrain/main/hd-vila-100m/src/download_videos.py -O /content/download_videos.py

!pip install datasets yt_dlp jsonlines

# from datasets import load_dataset
# dataset = load_dataset("camenduru/microsoft-XPretrain")
# dataset

In [ ]:
import datasets, json, jsonlines
from yt_dlp import YoutubeDL

!mkdir /content/videos

def download_video(id):
  ydl_opts = {'overwrites':True, 'format':'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4', 'outtmpl':f'/content/videos/{id}.mp4'}
  with YoutubeDL(ydl_opts) as ydl:
    ydl.download(id)
    return f"/content/videos/{id}.mp4"

dataset = datasets.load_dataset("camenduru/microsoft-XPretrain", split='train[:1%]')
first_10_data = list(dataset)[:10]
# print(first_10_data[0])

for dump in first_10_data:
  print(dump)
  json_string = json.dumps(dump)
  row = json.loads(json_string)
  download_id = row['video_id']
  print(download_id)
  try:
    download_video(download_id)
  except:
    pass

In [ ]:
json_string = json.dumps(first_10_data[0])
row = json.loads(json_string)
print(row)

In [ ]:
{
   "video_id":"tS3XvuOhbNo",
   "url":"https://www.youtube.com/watch?v=tS3XvuOhbNo",
   "clip":[
      {
         "clip_id":"tS3XvuOhbNo.0.mp4",
         "span":[
            "00:00:03.350",
            "00:00:13.440"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.1.mp4",
         "span":[
            "00:00:13.440",
            "00:00:26.130"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.2.mp4",
         "span":[
            "00:00:26.130",
            "00:00:31.140"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.3.mp4",
         "span":[
            "00:00:31.140",
            "00:00:39.629"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.4.mp4",
         "span":[
            "00:00:39.629",
            "00:00:45.420"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.5.mp4",
         "span":[
            "00:00:45.420",
            "00:01:00.149"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.6.mp4",
         "span":[
            "00:01:00.149",
            "00:01:14.040"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.7.mp4",
         "span":[
            "00:01:14.040",
            "00:01:25.290"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.8.mp4",
         "span":[
            "00:01:25.290",
            "00:01:35.460"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.9.mp4",
         "span":[
            "00:01:35.460",
            "00:01:54.109"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.10.mp4",
         "span":[
            "00:01:54.119",
            "00:02:10.200"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.11.mp4",
         "span":[
            "00:02:10.200",
            "00:02:16.319"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.12.mp4",
         "span":[
            "00:02:16.319",
            "00:02:23.250"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.13.mp4",
         "span":[
            "00:02:23.250",
            "00:02:58.709"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.14.mp4",
         "span":[
            "00:02:58.709",
            "00:03:04.829"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.15.mp4",
         "span":[
            "00:03:04.829",
            "00:03:14.340"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.16.mp4",
         "span":[
            "00:03:14.340",
            "00:03:22.470"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.17.mp4",
         "span":[
            "00:03:22.470",
            "00:03:29.639"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.18.mp4",
         "span":[
            "00:03:29.639",
            "00:03:39.139"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.19.mp4",
         "span":[
            "00:03:39.139",
            "00:03:47.910"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.20.mp4",
         "span":[
            "00:03:47.910",
            "00:03:54.120"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.21.mp4",
         "span":[
            "00:03:54.120",
            "00:04:05.780"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.22.mp4",
         "span":[
            "00:04:05.780",
            "00:04:13.400"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.23.mp4",
         "span":[
            "00:04:13.400",
            "00:04:21.320"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.24.mp4",
         "span":[
            "00:04:21.320",
            "00:04:29.000"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.25.mp4",
         "span":[
            "00:04:29.000",
            "00:04:37.490"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.26.mp4",
         "span":[
            "00:04:37.490",
            "00:04:48.590"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.27.mp4",
         "span":[
            "00:04:48.590",
            "00:04:54.110"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.28.mp4",
         "span":[
            "00:04:54.110",
            "00:05:06.260"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.29.mp4",
         "span":[
            "00:05:06.260",
            "00:05:12.110"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.30.mp4",
         "span":[
            "00:05:12.110",
            "00:05:17.180"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.31.mp4",
         "span":[
            "00:05:17.180",
            "00:05:32.930"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.32.mp4",
         "span":[
            "00:05:32.930",
            "00:05:45.170"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.33.mp4",
         "span":[
            "00:05:45.170",
            "00:05:55.250"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.34.mp4",
         "span":[
            "00:05:55.250",
            "00:06:05.990"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.35.mp4",
         "span":[
            "00:06:05.990",
            "00:06:16.550"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.36.mp4",
         "span":[
            "00:06:16.550",
            "00:06:22.070"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.37.mp4",
         "span":[
            "00:06:22.070",
            "00:06:29.570"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.38.mp4",
         "span":[
            "00:06:29.570",
            "00:06:36.650"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.39.mp4",
         "span":[
            "00:06:36.650",
            "00:06:44.540"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.40.mp4",
         "span":[
            "00:06:44.540",
            "00:06:49.940"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.41.mp4",
         "span":[
            "00:06:49.940",
            "00:06:56.870"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.42.mp4",
         "span":[
            "00:06:56.870",
            "00:07:09.440"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.43.mp4",
         "span":[
            "00:07:09.440",
            "00:07:16.640"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.44.mp4",
         "span":[
            "00:07:16.640",
            "00:07:28.160"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.45.mp4",
         "span":[
            "00:07:28.160",
            "00:07:33.950"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.46.mp4",
         "span":[
            "00:07:33.950",
            "00:07:41.150"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.47.mp4",
         "span":[
            "00:07:41.150",
            "00:07:52.320"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.48.mp4",
         "span":[
            "00:07:52.320",
            "00:08:01.350"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.49.mp4",
         "span":[
            "00:08:01.350",
            "00:08:13.140"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.50.mp4",
         "span":[
            "00:08:13.140",
            "00:08:21.270"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.51.mp4",
         "span":[
            "00:08:21.270",
            "00:08:29.460"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.52.mp4",
         "span":[
            "00:08:29.460",
            "00:08:43.270"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.53.mp4",
         "span":[
            "00:08:43.280",
            "00:08:49.080"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.54.mp4",
         "span":[
            "00:08:49.080",
            "00:08:58.470"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.55.mp4",
         "span":[
            "00:08:58.470",
            "00:09:04.860"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.56.mp4",
         "span":[
            "00:09:04.860",
            "00:09:10.980"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.57.mp4",
         "span":[
            "00:09:10.980",
            "00:09:20.160"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.58.mp4",
         "span":[
            "00:09:20.160",
            "00:09:29.850"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.59.mp4",
         "span":[
            "00:09:29.850",
            "00:09:46.040"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.60.mp4",
         "span":[
            "00:09:46.040",
            "00:09:58.339"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.61.mp4",
         "span":[
            "00:09:58.339",
            "00:10:04.430"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.62.mp4",
         "span":[
            "00:10:04.430",
            "00:10:14.120"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.63.mp4",
         "span":[
            "00:10:14.120",
            "00:10:30.860"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.64.mp4",
         "span":[
            "00:10:30.860",
            "00:10:40.189"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.65.mp4",
         "span":[
            "00:10:40.189",
            "00:10:51.079"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.66.mp4",
         "span":[
            "00:10:51.079",
            "00:11:06.889"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.67.mp4",
         "span":[
            "00:11:06.889",
            "00:11:17.000"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.68.mp4",
         "span":[
            "00:11:17.000",
            "00:11:22.910"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.69.mp4",
         "span":[
            "00:11:22.910",
            "00:11:29.240"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.70.mp4",
         "span":[
            "00:11:29.240",
            "00:11:36.620"
         ]
      },
      {
         "clip_id":"tS3XvuOhbNo.71.mp4",
         "span":[
            "00:11:36.620",
            "00:11:41.410"
         ]
      }
   ]
}

In [ ]:
!apt-get -y install -qq aria2

import os
from huggingface_hub import create_repo, upload_folder

model_url = "https://hdvila.blob.core.windows.net/dataset/hdvila100m.zip" 
model_name = "hdvila100m.zip" 
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {model_url} -d /content/models -o {model_name}

%cd /content/models
!unzip /content/models/hdvila100m.zip

In [ ]:
!python /content/download_videos.py --metafile /content/models/hdvila/hdvila_part0.jsonl